## Installs

In [ ]:
#%%capture
!pip install torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
!pip install pytorch_lightning==2.1
!pip install pandas librosa opencv-python matplotlib  #cupy-cuda110 
!pip install -U albumentations

## Imports

In [ ]:
import pandas as pd
import os
import torch
from IPython.display import display, Audio
import librosa
import math
import numpy as np
import scipy
from scipy import signal as sci_signal
import cv2
import matplotlib.pyplot as plt
import gc
import sys
import re
import numpy as np
from glob import glob
from tqdm import tqdm
import pytorch_lightning as pl
import albumentations as albu
from torchvision.models import efficientnet
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
import pandas as pd
import pandas.api.types
import sklearn.metrics
from sklearn.model_selection import StratifiedKFold

## Configuration

In [ ]:
class CONFIG:
    
    # == GENERAL ==
    seed = 1917                             # random seed
    device = 'cuda'                         # device to be used
    mix_precision = False                   # whether to use mixed-16 precision
    
    # == DATA ==
    use_holdout = False                                                  # Creates a holdout set
    holdout_path = '../../input/birdclef-2024/holdout_55bottom_20.0%_ratleq3_.csv'  # Path of files from holdout (generated with CreateHoldout.ipynb)
    holdout_noise_val = [0.01, 0.02]                                    # Adds noise to the holdout specs if using holdout ḿin_var, max_var]

    preprocess = True                                                  # If true, will recalculated all center specs for the training set
    preprocessed_data = 'VictorFeatures/'                                  # Path for processed data to be stores (Must put on .gitignore to not send to repo)
    checkpoint_dir = '../../input/efficient-net-1foldstrat-CEBAL'          # Checkpoints path (Must put on .gitignore to not send to repo)
    data_dir_2024 = '../../input/birdclef-2024'# root folder
    sr = 32000                              # sampling rate
    n_fft = 1095                            # NFFT of Spec.
    win_len = 412                           # WIN_SIZE of Spec.
    hop_len = 100                           # overlap of Spec.
    min_freq = 40                           # min frequency
    max_freq = 15000                        # max frequency
    
    # == MODEL ==
    model = 'efficientnet_b0'               # model architecture
    
    # == DATASET ==
    batch_size = 32                         # batch size of each step
    n_workers = 4                           # number of workers
    
    # == AUG ==
    USE_HORIZFLIP = True
    USE_XYMASKING = True                    # whether use XYMasking

    # == TRAINING ==
    folds = 3                               # n fold
    epochs = 1                              # max epochs
    lr = 1e-3                               # learning rate
    weight_decay = 1e-5                     # weight decay of optimizer
    visualize = True                        # whether to visualize data and batch

In [ ]:
!pwd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device, ', Using', torch.cuda.device_count(), 'GPU(s)')
pl.seed_everything(CONFIG.seed, workers=True)

## Kagle Score

In [ ]:
class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Version of macro-averaged ROC-AUC score that ignores all classes that have no true positive labels.
    '''
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    if not pandas.api.types.is_numeric_dtype(submission.values):
        bad_dtypes = {x: submission[x].dtype  for x in submission.columns if not pandas.api.types.is_numeric_dtype(submission[x])}
        raise ParticipantVisibleError(f'Invalid submission data types found: {bad_dtypes}')

    solution_sums = solution.sum(axis=0)
    scored_columns = list(solution_sums[solution_sums > 0].index.values)
    assert len(scored_columns) > 0

    return sklearn.metrics.roc_auc_score(solution[scored_columns].values, submission[scored_columns].values, average='macro')

## Dataframes

Define which data is gonna be used for both Train-Val and Holdout

In [ ]:
# READ 2024 TRAIN DATAFRAME AND ADD LABELS
df = pd.read_csv(f'{CONFIG.data_dir_2024}/train_metadata.csv')
df.head()
label_list = sorted(df['primary_label'].unique())
label_id_list = list(range(len(label_list)))
label2id = dict(zip(label_list, label_id_list))
id2label = dict(zip(label_id_list, label_list))
df = df[['primary_label', 'rating', 'filename']].copy() 
df['target'] = df.primary_label.map(label2id)
df['filepath'] = CONFIG.data_dir_2024 + '/train_audio/' + df.filename
df['name'] = df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])


# CREATE HOLDOUT
if CONFIG.use_holdout:
    holdout_df = pd.read_csv(CONFIG.holdout_path)
    holdout_df = df[df['name'].isin(holdout_df['name'])] 
    train_df = df[~df['name'].isin(holdout_df['name'])] 
    print('> #Holdout Samples: ', len(holdout_df))
    print('> #Training Samples:', len(train_df))
    print(holdout_df)
    print(train_df)
else:
    train_df = df
    print('> #Training Samples:', len(train_df))
    print(train_df)

## Preprocessing

Generates Image Spectrograms, for both train-val (audio center), and holdout (every 5 seconds of audio)

### Train-Val

In [ ]:
def preprocess_fn(filepath):

    # LOAD .OGG
    audio_data, _ = librosa.load(filepath, sr=CONFIG.sr)

    # CROP CENTER 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
    n_copy = math.ceil(5 * CONFIG.sr / len(audio_data))
    if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
    start_idx = int(len(audio_data) / 2 - 2.5 * CONFIG.sr)
    end_idx = int(start_idx + 5.0 * CONFIG.sr)
    input_audio = audio_data[start_idx:end_idx]

    # HANDLE NaNs
    mean_signal = np.nanmean(input_audio)
    input_audio = np.nan_to_num(input_audio, nan=mean_signal) if np.isnan(input_audio).mean() < 1 else np.zeros_like(input_audio)
    
    # SPECTROGRAM
    frequencies, times, spec_data = sci_signal.spectrogram(
        input_audio, 
        fs=CONFIG.sr, 
        nfft=CONFIG.n_fft, 
        nperseg=CONFIG.win_len, 
        noverlap=CONFIG.hop_len, 
        window='hann'
    )
    
    # FILTER LOWER AND HIGHER FREQUENCIES
    valid_freq = (frequencies >= CONFIG.min_freq) & (frequencies <= CONFIG.max_freq)
    spec_data = spec_data[valid_freq, :]
    
    # COMPUTE LOG SPEC
    spec_data = np.log10(spec_data + 1e-20)
    
    # MIN/MAX NORMALIZATION
    spec_data = spec_data - spec_data.min()
    spec_data = spec_data / spec_data.max()

    # SPEC TO IMAGE
    spec_data = cv2.resize(spec_data, (256, 256), interpolation=cv2.INTER_AREA)
    
    return spec_data

In [ ]:
# PROCESS TRAIN
if not CONFIG.preprocess:
    all_bird_data = np.load(f'{CONFIG.preprocessed_data}/spec_center_5sec_256_256.npy', allow_pickle=True).item()
else:
    all_bird_data = dict()
    for i, row_metadata in tqdm(train_df.iterrows()):
        
        # filepath to processed spec img
        processed_spec = preprocess_fn(row_metadata.filepath)

        all_bird_data[row_metadata.name] = processed_spec.astype(np.float32)

    # save to file
    np.save(os.path.join(CONFIG.preprocessed_data, f'spec_center_5sec_256_256.npy'), all_bird_data)

samples_per_class = train_df['target'].value_counts().sort_index().to_list()

### Holdout

In [ ]:
def oog2spec_via_scipy(audio_data):
    # handles NaNs
    mean_signal = np.nanmean(audio_data)
    audio_data = np.nan_to_num(audio_data, nan=mean_signal) if np.isnan(audio_data).mean() < 1 else np.zeros_like(audio_data)
    
    # to spec.
    frequencies, times, spec_data = sci_signal.spectrogram(
        audio_data, 
        fs=CONFIG.sr, 
        nfft=CONFIG.n_fft, 
        nperseg=CONFIG.win_len, 
        noverlap=CONFIG.hop_len, 
        window='hann'
    )
    
    # Filter frequency range
    valid_freq = (frequencies >= CONFIG.min_freq) & (frequencies <= CONFIG.max_freq)
    spec_data = spec_data[valid_freq, :]
    
    # Log
    spec_data = np.log10(spec_data + 1e-20)
    
    # min/max normalize
    spec_data = spec_data - spec_data.min()
    spec_data = spec_data / spec_data.max()
    
    return spec_data

In [ ]:
# PROCESS HOLDOUT
if not CONFIG.use_holdout:
    print('entrou')
    all_bird_data_holdout = dict()
    temp_df = pd.DataFrame()
    for i, row in tqdm(holdout_df.iterrows()):
        row_id = row['name']
        file_path = row['filepath']
        audio_data, _ = librosa.load(file_path, sr=CONFIG.sr)
        spec = oog2spec_via_scipy(audio_data)
        pad = 512 - (spec.shape[1] % 512)
        if pad > 0:
            spec = np.pad(spec, ((0,0), (0,pad)))
        spec = spec.reshape(512,-1,512).transpose([0, 2, 1])
        spec = cv2.resize(spec, (256, 256), interpolation=cv2.INTER_AREA)
        if len(spec.shape) == 3:
            for j in range(spec.shape[-1]):
                all_bird_data_holdout[f'{row_id}_{(j+1)*5}'] = spec[:, :, j]
                temp_row = row
                temp_row['name'] = f'{row_id}_{(j+1)*5}'
                temp_df = pd.concat([temp_df, temp_row.to_frame().transpose()])
        else:
            j=0
            all_bird_data_holdout[f'{row_id}_{(j+1)*5}'] = spec[:, :]
            temp_row = row
            temp_row['name'] = f'{row_id}_{(j+1)*5}'
            temp_df = pd.concat([temp_df, temp_row.to_frame().transpose()])
    holdout_df = temp_df
    print('> #Holdout Samples: ', len(all_bird_data_holdout))

## Dataset

In [ ]:
class BirdDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        metadata,
        data,
        augmentation=None,
        holdout_flag=False,
    ):
        super().__init__()
        self.metadata = metadata
        self.augmentation = augmentation
        self.data = data
        self.holdout_flag = holdout_flag
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, index):
        row_metadata = self.metadata.iloc[index]
        
        # load spec. data
        if self.holdout_flag:
            input_spec = self.data[row_metadata['name']]
        else:
            input_spec = self.data[row_metadata.name]
        # aug
        if self.augmentation is not None:
            input_spec = self.augmentation(image=input_spec)['image']
        
        # target
        target = row_metadata.target
        
        return torch.tensor(input_spec, dtype=torch.float32), torch.tensor(target, dtype=torch.long)

In [ ]:
def get_transforms(_type):
    
    if _type == 'train':
        return albu.Compose([
            albu.HorizontalFlip(0.5) if CONFIG.USE_XYMASKING else albu.NoOp(),
            albu.XYMasking(
                p=0.3,
                num_masks_x=(1, 3),
                num_masks_y=(1, 3),
                mask_x_length=(1, 10),
                mask_y_length=(1, 20),
            ) if CONFIG.USE_XYMASKING else albu.NoOp()
        ])
    elif _type == 'valid':
        return albu.Compose([])
    elif _type == 'test':
        return albu.Compose([
            albu.GaussNoise(var_limit=(CONFIG.holdout_noise_val[0], CONFIG.holdout_noise_val[1]), mean=0)
        ])

In [ ]:
def show_batch(ds, row=3, col=3):
    fig = plt.figure(figsize=(10, 10))
    img_index = np.random.randint(0, len(ds)-1, row*col)
    
    for i in range(len(img_index)):
        img, label = dummy_dataset[img_index[i]]
        
        if isinstance(img, torch.Tensor):
            img = img.detach().numpy()
        
        ax = fig.add_subplot(row, col, i + 1, xticks=[], yticks=[])
        ax.imshow(img, cmap='jet')
        ax.set_title(f'ID: {img_index[i]}; Target: {label}')
    
    plt.tight_layout()
    plt.show()

In [ ]:
dummy_dataset = BirdDataset(train_df, all_bird_data, get_transforms('train'))

test_input, test_target = dummy_dataset[0]
print(test_input.detach().numpy().shape)

show_batch(dummy_dataset)

del dummy_dataset
gc.collect()


In [64]:
if CONFIG.use_holdout:
    dummy_dataset = BirdDataset(holdout_df, all_bird_data_holdout, get_transforms('test'), True)

    test_input, test_target = dummy_dataset[0]
    print(test_input.detach().numpy().shape)

    show_batch(dummy_dataset)

    del dummy_dataset
    gc.collect()


## Cross-Validation

In [65]:
train_df

,primary_label,rating,filename,target,filepath,name
0,asbfly,5.0,asbfly/XC134896.ogg,0,../../input/birdclef-2024/train_audio/asbfly/X...,asbfly-XC134896
1,asbfly,2.5,asbfly/XC164848.ogg,0,../../input/birdclef-2024/train_audio/asbfly/X...,asbfly-XC164848
2,asbfly,2.5,asbfly/XC175797.ogg,0,../../input/birdclef-2024/train_audio/asbfly/X...,asbfly-XC175797
3,asbfly,4.0,asbfly/XC207738.ogg,0,../../input/birdclef-2024/train_audio/asbfly/X...,asbfly-XC207738
4,asbfly,4.0,asbfly/XC209218.ogg,0,../../input/birdclef-2024/train_audio/asbfly/X...,asbfly-XC209218
...,...,...,...,...,...,...
24454,zitcis1,5.0,zitcis1/XC845747.ogg,181,../../input/birdclef-2024/train_audio/zitcis1/...,zitcis1-XC845747
24455,zitcis1,4.0,zitcis1/XC845817.ogg,181,../../input/birdclef-2024/train_audio/zitcis1/...,zitcis1-XC845817
24456,zitcis1,4.0,zitcis1/XC856176.ogg,181,../../input/birdclef-2024/train_audio/zitcis1/...,zitcis1-XC856176
24457,zitcis1,4.5,zitcis1/XC856723.ogg,181,../../input/birdclef-2024/train_audio/zitcis1/...,zitcis1-XC856723


In [66]:
kf = StratifiedKFold(n_splits=CONFIG.folds, shuffle=True, random_state=CONFIG.seed)
train_df['fold'] = 0
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df, y=train_df['primary_label'])):
    for idx in val_idx:
        train_df.iloc[idx, train_df.columns.get_loc('fold') ] = fold

/tmp/ipykernel_105378/1560425894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['fold'] = 0


## Model

In [67]:
class EffNet(nn.Module):
    
    def __init__(self, model_type, n_classes, pretrained=True):
        super().__init__()
        
        if model_type == 'efficientnet_b0':
            if pretrained: weights = efficientnet.EfficientNet_B0_Weights.DEFAULT
            else: weights = None
            self.base_model = efficientnet.efficientnet_b0(weights=weights)
        elif model_type == 'efficientnet_b1':
            if pretrained: weights = efficientnet.EfficientNet_B1_Weights.DEFAULT
            else: weights = None
            self.base_model = efficientnet.efficientnet_b1(weights=weights)
        elif model_type == 'efficientnet_b2':
            if pretrained: weights = efficientnet.EfficientNet_B2_Weights.DEFAULT
            else: weights = None
            self.base_model = efficientnet.efficientnet_b2(weights=weights)
        elif model_type == 'efficientnet_b3':
            if pretrained: weights = efficientnet.EfficientNet_B3_Weights.DEFAULT
            else: weights = None
            self.base_model = efficientnet.efficientnet_b3(weights=weights)
        else:
            raise ValueError('model type not supported')
        
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, n_classes, dtype=torch.float32)
    
    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.cat([x, x, x], dim=3).permute(0, 3, 1, 2)
        return self.base_model(x)

In [68]:
dummy_model = EffNet(CONFIG.model, n_classes=len(label_list))
dummy_input = torch.randn(2, 256, 256)
print(dummy_model(dummy_input).shape)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 86.9MB/s]


torch.Size([2, 182])


## Lightning Module

In [69]:
def focal_loss(logits, labels, alpha=None, gamma=2):
    """Compute the focal loss between `logits` and the ground truth `labels`.
    Focal loss = -alpha_t * (1-pt)^gamma * log(pt)
    where pt is the probability of being classified to the true class.
    pt = p (if true class), otherwise pt = 1 - p. p = sigmoid(logit).
    Args:
      logits: A float tensor of size [batch, num_classes].
      labels: A float tensor of size [batch, num_classes].
      alpha: A float tensor of size [batch_size]
        specifying per-example weight for balanced cross entropy.
      gamma: A float scalar modulating loss from hard and easy examples.
    Returns:
      focal_loss: A float32 scalar representing normalized total loss.
    """
    bc_loss = torch.nn.functional.binary_cross_entropy_with_logits(input=logits, target=labels, reduction="none")

    if gamma == 0.0:
        modulator = 1.0
    else:
        modulator = torch.exp(-gamma * labels * logits - gamma * torch.log(1 + torch.exp(-1.0 * logits)))

    loss = modulator * bc_loss

    if alpha is not None:
        weighted_loss = alpha * loss
        focal_loss = torch.sum(weighted_loss)
    else:
        focal_loss = torch.sum(loss)

    focal_loss /= torch.sum(labels)
    return focal_loss

class BalancedCrossEntropyLoss(torch.nn.Module):
    def __init__(
        self,
        samples_per_class=None,
        loss_type: str = "cross_entropy",
        beta: float = 0.999,
        fl_gamma=2,
        class_balanced=True,
    ):
        """
        Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
        Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
        where Loss is one of the standard losses used for Neural Networks.
        reference: https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
        
        Example usage:
        
            # outputs and labels
            logits = torch.tensor([[0.78, 0.1, 0.05]]) # 1 batch, 3 class
            labels = torch.tensor([0]) # 1 batch

            # number of samples per class in the training dataset
            samples_per_class = [30, 100, 25] # 30, 100, 25 samples for labels 0, 1 and 2, respectively

            # class-balanced focal loss
            focal_loss = Loss(
                loss_type="focal_loss",
                samples_per_class=samples_per_class,
                class_balanced=True
            )
            
            loss = focal_loss(logits, labels)
        
        
        Args:
            loss_type: string. One of "focal_loss", "cross_entropy",
                "binary_cross_entropy", "softmax_binary_cross_entropy".
            beta: float. Hyperparameter for Class balanced loss.
            fl_gamma: float. Hyperparameter for Focal loss.
            samples_per_class: A python list of size [num_classes].
                Required if class_balance is True.
            class_balanced: bool. Whether to use class balanced loss.
        Returns:
            BalancedCrossEntropyLoss instance
        """
        super(BalancedCrossEntropyLoss, self).__init__()

        if class_balanced is True and samples_per_class is None:
            raise ValueError("samples_per_class cannot be None when class_balanced is True")

        self.loss_type = loss_type
        self.beta = beta
        self.fl_gamma = fl_gamma
        self.samples_per_class = samples_per_class
        self.class_balanced = class_balanced

    def forward(
        self,
        logits: torch.tensor,
        labels: torch.tensor,
    ):
        """
        Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
        Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
        where Loss is one of the standard losses used for Neural Networks.
        Args:
            logits: A float tensor of size [batch, num_classes].
            labels: An int tensor of size [batch].
        Returns:
            cb_loss: A float tensor representing class balanced loss
        """

        batch_size = logits.size(0)
        num_classes = logits.size(1)
        labels_one_hot = torch.nn.functional.one_hot(labels, num_classes).float()

        if self.class_balanced:
            effective_num = 1.0 - np.power(self.beta, self.samples_per_class)
            weights = (1.0 - self.beta) / np.array(effective_num)
            weights = weights / np.sum(weights) * num_classes
            weights = torch.tensor(weights, device=logits.device).float()

            if self.loss_type != "cross_entropy":
                weights = weights.unsqueeze(0)
                weights = weights.repeat(batch_size, 1) * labels_one_hot
                weights = weights.sum(1)
                weights = weights.unsqueeze(1)
                weights = weights.repeat(1, num_classes)
        else:
            weights = None

        if self.loss_type == "focal_loss":
            cb_loss = focal_loss(logits, labels_one_hot, alpha=weights, gamma=self.fl_gamma)
        elif self.loss_type == "cross_entropy":
            cb_loss = torch.nn.functional.cross_entropy(input=logits, target=labels_one_hot, weight=weights)
        elif self.loss_type == "binary_cross_entropy":
            cb_loss = torch.nn.functional.binary_cross_entropy_with_logits(input=logits, target=labels_one_hot, weight=weights)
        elif self.loss_type == "softmax_binary_cross_entropy":
            pred = logits.softmax(dim=1)
            cb_loss = torch.nn.functional.binary_cross_entropy(input=pred, target=labels_one_hot, weight=weights)
        return cb_loss


In [70]:
class BirdModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        # == backbone ==
        self.backbone = EffNet(CONFIG.model, n_classes=len(label_list))
        
        # == loss function ==
        self.loss_fn = BalancedCrossEntropyLoss(samples_per_class=samples_per_class)
        
        # == record ==
        self.validation_step_outputs = []
        
    def forward(self, images):
        return self.backbone(images)
    
    def configure_optimizers(self):
        
        # == define optimizer ==
        model_optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr=CONFIG.lr,
            weight_decay=CONFIG.weight_decay
        )
        
        # == define learning rate scheduler ==
        lr_scheduler = CosineAnnealingWarmRestarts(
            model_optimizer,
            T_0=CONFIG.epochs,
            T_mult=1,
            eta_min=1e-6,
            last_epoch=-1
        )
        
        return {
            'optimizer': model_optimizer,
            'lr_scheduler': {
                'scheduler': lr_scheduler,
                'interval': 'epoch',
                'monitor': 'val_loss',
                'frequency': 1
            }
        }
    
    def training_step(self, batch, batch_idx):
        
        # == obtain input and target ==
        image, target = batch
        image = image.to(self.device)
        target = target.to(self.device)
        
        # == pred ==
        y_pred = self(image)
        
        # == compute loss ==
        train_loss = self.loss_fn(y_pred, target)
        
        # == record ==
        self.log('train_loss', train_loss, True)
        
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        
        # == obtain input and target ==
        image, target = batch
        image = image.to(self.device)
        target = target.to(self.device)
        
        # == pred ==
        with torch.no_grad():
            y_pred = self(image)
            
        self.validation_step_outputs.append({"logits": y_pred, "targets": target})
        
    def train_dataloader(self):
        return self._train_dataloader

    def validation_dataloader(self):
        return self._validation_dataloader
    
    def on_validation_epoch_end(self):
        
        # = merge batch data =
        outputs = self.validation_step_outputs
        
        output_val = nn.Softmax(dim=1)(torch.cat([x['logits'] for x in outputs], dim=0)).cpu().detach()
        target_val = torch.cat([x['targets'] for x in outputs], dim=0).cpu().detach()
        
        # = compute validation loss =
        val_loss = self.loss_fn(output_val, target_val)
        
        # target to one-hot
        target_val = torch.nn.functional.one_hot(target_val, len(label_list))
        
        # = val with ROC AUC =
        gt_df = pd.DataFrame(target_val.numpy().astype(np.float32), columns=label_list)
        pred_df = pd.DataFrame(output_val.numpy().astype(np.float32), columns=label_list)
        
        gt_df['id'] = [f'id_{i}' for i in range(len(gt_df))]
        pred_df['id'] = [f'id_{i}' for i in range(len(pred_df))]
        
        val_score = score(gt_df, pred_df, row_id_column_name='id')
        
        self.log("val_score", val_score, True)
        
        # clear validation outputs
        self.validation_step_outputs = list()
        
        return {'val_loss': val_loss, 'val_score': val_score}

## Training Loop

In [71]:
def predict(data_loader, model):
    model.to(CONFIG.device)
    model.eval()
    predictions = []
    gts = []
    for batch in tqdm(data_loader):
        with torch.no_grad():
            x, y = batch
            x = x.cuda()
            outputs = model(x)
            outputs = nn.Softmax(dim=1)(outputs)
        predictions.append(outputs.detach().cpu())
        gts.append(y.detach().cpu())
    
    predictions = torch.cat(predictions, dim=0).cpu().detach()
    gts = torch.cat(gts, dim=0).cpu().detach()
    gts = torch.nn.functional.one_hot(gts, len(label_list))
    return predictions.numpy().astype(np.float32), gts.numpy().astype(np.float32)

In [72]:
def run_training(fold_id, total_df):
    print('================================================================')
    print(f"==== Running training for fold {fold_id} ====")
    
    # == create dataset and dataloader ==
    train_df = total_df[total_df['fold'] != fold_id].copy()
    valid_df = total_df[total_df['fold'] == fold_id].copy()
    
    print(f'Train Samples: {len(train_df)}')
    print(f'Valid Samples: {len(valid_df)}')
    
    train_ds = BirdDataset(train_df, all_bird_data, get_transforms('train'), False)
    val_ds = BirdDataset(valid_df, all_bird_data, get_transforms('valid'), False)
    
    train_dl = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.n_workers,
        pin_memory=True,
        persistent_workers=True
    )
    
    val_dl = torch.utils.data.DataLoader(
        val_ds,
        batch_size=CONFIG.batch_size * 2,
        shuffle=False,
        num_workers=CONFIG.n_workers,
        pin_memory=True,
        persistent_workers=True
    )
    
    # == init model ==
    bird_model = BirdModel()
    
    # == init callback ==
    checkpoint_callback = ModelCheckpoint(monitor='val_score',
                                          dirpath=CONFIG.checkpoint_dir,
                                          save_top_k=1,
                                          save_last=False,
                                          save_weights_only=True,
                                          filename=f"fold_{fold_id}",
                                          mode='max')
    callbacks_to_use = [checkpoint_callback, TQDMProgressBar(refresh_rate=1)]
    
    # == init trainer ==
    trainer = pl.Trainer(
        max_epochs=CONFIG.epochs,
        val_check_interval=0.5,
        callbacks=callbacks_to_use,
        enable_model_summary=False,
        accelerator="gpu",
        deterministic=True,
        precision='16-mixed' if CONFIG.mix_precision else 32,
    )
    
    # == Training ==
    trainer.fit(bird_model, train_dataloaders=train_dl, val_dataloaders=val_dl)
    
    # == Prediction ==
    best_model_path = checkpoint_callback.best_model_path
    weights = torch.load(best_model_path)['state_dict']
    bird_model.load_state_dict(weights)
    
    preds, gts = predict(val_dl, bird_model)
    
    # = create dataframe =
    pred_df = pd.DataFrame(preds, columns=label_list)
    pred_df['id'] = np.arange(len(pred_df))
    gt_df = pd.DataFrame(gts, columns=label_list)
    gt_df['id'] = np.arange(len(gt_df))
    
    # = compute score =
    val_score = score(gt_df, pred_df, row_id_column_name='id')
    
    # == save to file ==
    pred_cols = [f'pred_{t}' for t in label_list]
    valid_df = pd.concat([valid_df.reset_index(), pd.DataFrame(np.zeros((len(valid_df), len(label_list)*2)).astype(np.float32), columns=label_list+pred_cols)], axis=1)
    valid_df[label_list] = gts
    valid_df[pred_cols] = preds
    valid_df.to_csv(f"{CONFIG.checkpoint_dir}/pred_df_f{fold_id}.csv", index=False)
    
    return preds, gts, val_score

In [73]:
def run_holdout(f, holdout_df):
    bird_model = BirdModel()
    weights = torch.load(os.path.join(CONFIG.checkpoint_dir, 'fold_' + str(f) + '.ckpt'), map_location=torch.device('cpu'))['state_dict']
    bird_model.load_state_dict(weights)    
    test_dataset = BirdDataset(holdout_df, all_bird_data_holdout, get_transforms('test'), True)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size * 2,
        shuffle=False,
        num_workers=CONFIG.n_workers,
        pin_memory=True,
        persistent_workers=True
    )
    preds, gts = predict(test_loader, bird_model)
    pred_df = pd.DataFrame(preds, columns=label_list)
    pred_df['id'] = np.arange(len(pred_df))
    gt_df = pd.DataFrame(gts, columns=label_list)
    gt_df['id'] = np.arange(len(gt_df))
    holdout_score = score(gt_df, pred_df, row_id_column_name='id')
    pred_cols = [f'pred_{t}' for t in label_list]
    holdout_df = pd.concat([holdout_df.reset_index(), pd.DataFrame(np.zeros((len(holdout_df), len(label_list)*2)).astype(np.float32), columns=label_list+pred_cols)], axis=1)
    holdout_df[label_list] = gts
    holdout_df[pred_cols] = preds
    holdout_df.to_csv(f"{CONFIG.checkpoint_dir}/holdout_pred_df_f{f}.csv", index=False)
    return preds, gts, holdout_score

## RUN

In [78]:
!export CUDA_LAUNCH_BLOCKING=1
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


# training
torch.set_float32_matmul_precision('high')

# record
fold_val_score_list = list()
oof_df = train_df.copy()
pred_cols = [f'pred_{t}' for t in label_list]
oof_df = pd.concat([oof_df, pd.DataFrame(np.zeros((len(oof_df), len(pred_cols)*2)).astype(np.float32), columns=label_list+pred_cols)], axis=1)

if CONFIG.use_holdout:
    # record
    fold_test_score_list = list()
    holdout_df.index = range(len(holdout_df))
    ood_df = holdout_df.copy()
    pred_cols = [f'pred_{t}' for t in label_list]
    hold_idx = holdout_df.index
    ood_df = pd.concat([ood_df, pd.DataFrame(np.zeros((len(ood_df), len(pred_cols)*2)).astype(np.float32), columns=label_list+pred_cols)], axis=1)
    hold_predictions = list()

for f in range(CONFIG.folds):
    
    # get validation index
    val_idx = list(train_df[train_df['fold'] == f].index)
    
    # main loop of f-fold
    val_preds, val_gts, val_score = run_training(f, train_df)
    oof_df.loc[val_idx, label_list] = val_gts
    oof_df.loc[val_idx, pred_cols] = val_preds
    fold_val_score_list.append(val_score)

    if CONFIG.use_holdout:
        # Holdout Fold Inference
        hold_preds, hold_gts, hold_score = run_holdout(f, holdout_df)
        ood_df.loc[hold_idx, label_list] = hold_gts
        ood_df.loc[hold_idx, pred_cols] = hold_preds
        fold_test_score_list.append(hold_score)
        hold_predictions.append(hold_preds)

for idx, val_score in enumerate(fold_val_score_list):
    print(f'Fold {idx} Val Score: {val_score:.5f}')
oof_df.to_csv(f"{CONFIG.checkpoint_dir}/oof_pred.csv", index=False)

# Holdout Score
if CONFIG.use_holdout:

    for idx, hold_score in enumerate(fold_test_score_list):
        print(f'Fold {idx} Holdout Score: {hold_score:.5f}')
    
    ood_df.to_csv(f"{CONFIG.checkpoint_dir}/ood_pred.csv", index=False)

    ensemble_df = holdout_df.copy()
    pred_cols = [f'pred_{t}' for t in label_list]
    ensemble_df = pd.concat([ensemble_df, pd.DataFrame(np.zeros((len(ensemble_df), len(pred_cols)*2)).astype(np.float32), columns=label_list+pred_cols)], axis=1)
    ensemble_predictions = np.mean(hold_predictions, axis=0)
    pred_df = pd.DataFrame(ensemble_predictions, columns=label_list)
    pred_df['id'] = np.arange(len(pred_df))
    gt_df = pd.DataFrame(hold_gts, columns=label_list)
    gt_df['id'] = np.arange(len(gt_df))
    holdout_score = score(gt_df, pred_df, row_id_column_name='id')
    print('Ensemble Score', holdout_score)
    ensemble_df.loc[hold_idx, label_list] = hold_gts
    ensemble_df.loc[hold_idx, pred_cols] = ensemble_predictions
    ensemble_df.to_csv(f"{CONFIG.checkpoint_dir}/holdout_ensemble_result.csv", index=False)

==== Running training for fold 0 ====
Train Samples: 16131
Valid Samples: 8066


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.